In [1]:
from mpl_toolkits.mplot3d import Axes3D
from rtree import index
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [2]:
%store -r mc_data

In [3]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        xlayer_model = load_model('./models/xlayer_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        track_classifier = load_model('./models/track_classifier.h5')

In [5]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event


In [6]:
mc_hits = mc_data.values

In [7]:
%store -r zLays
%store -r max_x
%store -r min_x
%store -r events

In [8]:
%store -r idx

In [9]:
print(events)

[15837 15838 15839 15840 15841 15842 15843 15844 15845 15846 15847 15848
 15849 15850 15851 15852 15853 15854 15855 15856 15857 15858 15859 15860
 15861 15862 15863 15864 15865 15866 15867 15868 15869 15870 15871 15872
 15873 15874 15875 15876 15877 15878 15879 15880 15881 15882 15883 15884
 15885 15886 15887 15888 15889 15890 15891 15892 15893 15894 15895 15896
 15897 15898 15899 15900 15901 15902 15903 15904 15905 15906 15907 15908
 15909 15910 15911 15912 15913 15914 15915 15916 15917 15918 15919 15920
 15921 15922 15923 15924 15925 15926 15927 15928 15929 15930 15931 15932
 15933 15934 15935 15936 15937 15938 15939 15940 15941 17909 17910 17911
 17912 17913 17914 17915 17916 17917 17918 17919 17920 17921 17922 17923
 17924 17925 17926 17927 17928 17929 17930 17931 17932 17933 17934 17935
 17936 17937 17938 17939 17940 17941 17942 17943 17944 17945 17946 17947
 17948 17949 17950 17951 17952 17953 17954 17955 17956 17957 17958 17959
 17960 17961 17962 17963 17964 17965 17966 17967 17

In [8]:
event_threshold = 15840

In [23]:
hits_total = mc_hits[mc_hits[:,13] > event_threshold]
print("Total hits with threshold %d" % (len(hits_total)))
event = 0
idx = {}
actual = 0
for hit in hits_total:
    if (not hit[13] in idx):
        p = index.Property()
        p.dimension = 3
        idx[hit[13]] = index.Index(properties=p, interleaved=False)
    
    if (len(hit[5]) > 1):
        for i in range(0,len(hit[5])):
            idx[hit[13]].insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), float(hit[5][i]))

Total hits with threshold 199144


In [12]:
%store idx

Stored 'idx' (dict)


In [24]:
X_classifier = []
Y_classifier = []

total_found = 0
total = 0
long_range = 300

for particle in hits_total:
    
    if (len(particle[5]) > 1):
        
        found_bool = 0
        found_layers = np.zeros(4)
        # Extract only 12 hits, as some of them are really close
        zone = particle[12][0]
        X_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
            else:
                X_final[actual].append(particle[5][i])
                
        if (not 0 in X_final):
            continue
            
        found_layers[0] = X_final[0][0]
            
        predicted = (xlayer_model.predict(np.array([X_final[0][0], 0]).reshape(1,-1)))[0]
        
        if (3 in X_final):
            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, 3, 3, zone, zone),objects='raw'))
            found_layer3 = sorted(found, key=lambda x: np.abs(x - predicted[2]))[:20]

            for i in range(0,len(found_layer3)):
                if (found_layer3[i] in X_final[3]):
                    found_bool += 1
                    found_layers[3] = found_layer3[i]
                    break
            
        if (2 in X_final):
            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, 2, 2, zone,zone),objects='raw'))
            found_layer2 = sorted(found, key=lambda x: np.abs(x - predicted[1]))[:20]

            for i in range(0,len(found_layer2)):
                if (found_layer2[i] in X_final[2]):
                    found_bool += 1
                    found_layers[2] = found_layer2[i]
                    break
        
        if (1 in X_final):
            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, 1, 1,zone,zone),objects='raw'))
            found_layer1 = sorted(found, key=lambda x: np.abs(x - predicted[0]))[:20]

            for i in range(0,len(found_layer1)):
                if (found_layer1[i] in X_final[1]):
                    found_bool += 1
                    found_layers[1] = found_layer1[i]
                    break
            
        if (found_bool == 3):
            fake_track = np.array([X_final[0][0], random.choice(found_layer1),random.choice(found_layer2),random.choice(found_layer3)])
            true_track = found_layers
            
            retries = 0
            
            while (fake_track == true_track).all() and retries != 4:
                fake_track = np.array([X_final[0][0], random.choice(found_layer1),random.choice(found_layer2),random.choice(found_layer3)])
                retries += 1
                
            total_found += 1
            
            if (retries == 4):
                X_classifier.append(true_track)
                Y_classifier.append(1)
            else:
                X_classifier.append(true_track)
                Y_classifier.append(1)
                X_classifier.append(fake_track)
                Y_classifier.append(0)
            
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

Mean found 0.858799
Total 126125


In [34]:
%store X_classifier
%store Y_classifier

Stored 'X_classifier' (list)
Stored 'Y_classifier' (list)


In [28]:
track_classifier = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
track_classifier.add(Dense(12, activation='relu', input_dim=4))
track_classifier.add(Dense(8, activation='relu'))
track_classifier.add(Dense(1, activation='sigmoid'))
track_classifier.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_classifier), np.array(Y_classifier))
print("X Train length: %d" % len(X_train))
print("X Test Length %d" % len(X_test))
print("Y Train length: %d" % len(y_train))
print("Y Test length: %d" % len(y_test))

X Train length: 162295
X Test Length 54099
Y Train length: 162295
Y Test length: 54099


In [31]:
track_classifier.fit(X_train, y_train, epochs=15, batch_size=200, verbose=1, validation_split=0.1)

Train on 146065 samples, validate on 16230 samples
Epoch 1/15
146065/146065 [==============================] - 2s 15us/sample - loss: 2.2602 - acc: 0.5898 - val_loss: 1.0142 - val_acc: 0.5472
Epoch 2/15
146065/146065 [==============================] - 2s 14us/sample - loss: 0.8099 - acc: 0.6086 - val_loss: 0.7051 - val_acc: 0.6924
Epoch 3/15
146065/146065 [==============================] - 2s 14us/sample - loss: 0.7237 - acc: 0.6298 - val_loss: 0.8280 - val_acc: 0.5725
Epoch 4/15
146065/146065 [==============================] - 2s 14us/sample - loss: 0.6944 - acc: 0.6530 - val_loss: 0.8284 - val_acc: 0.6381
Epoch 5/15
146065/146065 [==============================] - 2s 14us/sample - loss: 0.6603 - acc: 0.6841 - val_loss: 0.5641 - val_acc: 0.7723
Epoch 6/15
146065/146065 [==============================] - 2s 14us/sample - loss: 0.4584 - acc: 0.8136 - val_loss: 0.3540 - val_acc: 0.8653
Epoch 7/15
146065/146065 [==============================] - 2s 14us/sample - loss: 0.3301 - acc: 0.8793

In [32]:
score, acc = track_classifier.evaluate(X_test, y_test, batch_size=25)
print('Test score:', score)
print('Test accuracy:', acc)

54099/54099 [==============================] - 2s 44us/sample - loss: 0.1813 - acc: 0.9454
Test score: 0.18125923787621864
Test accuracy: 0.9454149


In [33]:
track_classifier.save("./models/track_classifier.h5")

In [48]:
import datetime as dt
X_test = []

total_found = 0
total = 0
long_range = 300

hits_total = mc_hits[mc_hits[:,13] < event_threshold]

for particle in hits_total:
    
    if (len(particle[5]) > 1):
        
        found_bool = 0
        found_layers = np.zeros(3)
        # Extract only 12 hits, as some of them are really close
        zone = particle[12][0]
        X_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
            else:
                X_final[actual].append(particle[5][i])
                
        if (not 0 in X_final):
            continue
            
        predicted = (xlayer_model.predict(np.array([X_final[0][0], 0]).reshape(1,-1)))[0]
        
        print(predicted)
        
        if (4 in X_final):
            found = list(idx[particle[13]].intersection((predicted[3] - long_range, predicted[3] + long_range, 4, 4,zone,zone),objects='raw'))
            found_layer4 = sorted(found, key=lambda x: np.abs(x - predicted[3]))[:10]
            found_bool += 1
        
        if (3 in X_final):
            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, 3, 3, zone, zone),objects='raw'))
            found_layer3 = sorted(found, key=lambda x: np.abs(x - predicted[2]))[:10]
            found_bool += 1
            
        if (2 in X_final):
            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, 2, 2, zone,zone),objects='raw'))
            found_layer2 = sorted(found, key=lambda x: np.abs(x - predicted[1]))[:10]
            found_bool += 1
        
        if (1 in X_final):
            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, 1, 1,zone,zone),objects='raw'))
            found_layer1 = sorted(found, key=lambda x: np.abs(x - predicted[0]))[:10]
            found_bool += 1
            
        if (found_bool == 4):
            retries = 0
            
            valid_list = []
            max_val = 0
            for h4 in found_layer4:
                for h3 in found_layer3:
                    for h2 in found_layer2:
                        for h1 in found_layer1:
                            pred = (track_classifier.predict(np.array([X_final[0][0], h1, h2, h3, h4]).reshape(1,-1))[0])[0]
                            if pred > max_val:
                                valid = [X_final[0][0], h1, h2, h3, h4]
                                max_val = pred
            
            if valid[1] in X_final[1] and valid[2] in X_final[2] and valid[3] in X_final[3] and valid[4] in X_final[4]:
                total_found += 1
                
            print(total/354)
            print(total_found/354)
            #print(total_found/354)
            #print(total_val)
            #print('\n')
            
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

[1344.3711 1331.2653 1353.7424 1449.1339]
0.0
0.0
[261.86023 264.366   268.9563  289.13846]
0.002824858757062147
0.0
[-244.05386 -283.24338 -305.46356 -349.80124]
0.005649717514124294
0.0
[139.9398  144.20387 146.77966 158.49117]
0.00847457627118644
0.0
[1401.4517 1387.5227 1410.9429 1510.3002]


KeyboardInterrupt: 